In [1]:
import numpy as np

import sys, os
# sys.path.append('./pyFM/') # UNCOMMENT THIS LINE IF YOU HAVE NOT INSTALLED pyFM (pip install pyfmaps)
sys.path.append('./VisualizationTools/')  # This is for visualization purposes only, not necessary to run the algorithms

import VisualizationTools as plu
import pyFM

from pyFM.mesh import TriMesh
import pyFM.spectral as spectral

import DiscreteOpt_legacy

In [2]:
def generate_init_map(mesh1, mesh2, k):
    
    FM_12 = spectral.mesh_p2p_to_FM(np.arange(mesh2.n_vertices), mesh1, mesh2, dims=k)
    p2p_21_init = spectral.mesh_FM_to_p2p(FM_12, mesh1, mesh2, n_jobs=-1)
    return p2p_21_init

# LOAD DATA 

In [3]:
mesh1 = TriMesh("data/tr_reg_000.off", area_normalize=True, center=True).process(intrinsic=True)
mesh2 = TriMesh("data/tr_reg_024.off", area_normalize=True, center=True).process(intrinsic=True)

# GENERATE INITIAL MAP 

In [4]:
p2p_12_init = generate_init_map(mesh2, mesh1, 10)
p2p_21_init = generate_init_map(mesh1, mesh2, 10)

# FIT MODEL 

In [6]:
model = DiscreteOpt_legacy.SmoothDiscreteOptimization(mesh1, mesh2)
model._set_params("rhm")  # Select "rhm" or "exact" usually. You can try "arap" and "nicp" also

In [14]:
model.opt_params

{'k_init': 10,
 'nit': 10,
 'step': 5,
 'n_inner': 5,
 'k_max': None,
 'log_space': True,
 'sm_weight_range': [0.1, 1.0]}

In [7]:
model.fit(p2p_21=p2p_21_init, p2p_12=p2p_12_init, n_jobs=-1, verbose=True)

  0%|          | 0/11 [00:00<?, ?it/s]

In [8]:
plu.plot_p2p(mesh1, mesh2, model.p2p_21, pretty=True)

/Users/robinmagnet/opt/miniconda3/envs/py39/lib/python3.9/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [15]:
plu.plot(TriMesh(mesh1.vertices[model.p2p_21], mesh2.faces))

/Users/robinmagnet/opt/miniconda3/envs/py39/lib/python3.9/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.021894…